Задание:

*   используйте данные MovieLens,
*   можно использовать любые модели из пакета SURPRISE,
*   получите RMSE на тестовом сете 0,87 и ниже.







Скачиваем пакет Surprise

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357282 sha256=68ed5688485436280f730078ceddc55a3d34b10cdf421c57c41d8a922fc82f19
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from tqdm import tqdm
from surprise.model_selection import cross_validate


Скачиваем и загружаем необходимые таблицы из датасета MovieLens.

In [ ]:
!wget 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip' -O ml-latest-small.zip

--2024-11-01 05:09:07--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   917KB/s    in 1.0s    

2024-11-01 05:09:09 (917 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [ ]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

Объединяем таблицы с информацией о фильмах и рейтингами в одну.

In [ ]:
mov_with_rat = movies.merge(ratings, on='movieId').reset_index(drop=True)
mov_with_rat.dropna(inplace=True)
mov_with_rat.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


Для дальнейшей работы создаем таблицу без лишней информации.

In [ ]:
dataset = pd.DataFrame({
    'uid': mov_with_rat.userId,
    'iid': mov_with_rat.title,
    'rating': mov_with_rat.rating
})

In [ ]:
dataset

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5
...,...,...,...
100831,184,Black Butler: Book of the Atlantic (2017),4.0
100832,184,No Game No Life: Zero (2017),3.5
100833,184,Flint (2017),3.5
100834,184,Bungo Stray Dogs: Dead Apple (2018),3.5


In [ ]:
len(dataset.iid.unique())

9719

Обрабатываем имеющиеся данные для определения формата данных для Surprise, диапазона оценок и для упрощения процесса обучения.

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Сначала будем обучать алгоритмы User-based коллаборативной фильтрации (KNNBasic, KNNWithMeans, KNNBaseline при 'user_based': True), затем для каждого алгоритма рассчитаем метрику RMSE.

In [ ]:
user_res = {}

In [ ]:
user_algo = {
    'KNNBasic': KNNBasic(sim_options={'user_based': True}),
    'KNNWithMeans': KNNWithMeans(sim_options={'user_based': True}),
    'KNNBaseline': KNNBaseline(sim_options={'user_based': True})
}

In [ ]:
for algo_name, algo in user_algo.items():
  result = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
  user_res[algo_name] = result['test_rmse'].mean()

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity ma

In [ ]:
print('User-based collaborative filtering')
for algo_name, rmse in user_res.items():
  print(f"Средний RMSE для {algo_name}: {round(rmse,4)}")

User-based collaborative filtering
Средний RMSE для KNNBasic: 0.9462
Средний RMSE для KNNWithMeans: 0.8962
Средний RMSE для KNNBaseline: 0.8741


Значения RMSE получились выше требуемых 0,87, поэтому алгоритмы User-based коллаборативной фильтрации не подходят.

Далее будем обучать алгоритмы Item-based коллаборативной фильтрации (KNNBasic, KNNWithMeans, KNNBaseline при 'user_based': False), для каждого алгоритма рассчитаем метрику RMSE.

In [ ]:
item_res = {}

In [ ]:
item_algo = {
    'KNNBasic': KNNBasic(sim_options={'user_based': False}),
    'KNNWithMeans': KNNWithMeans(sim_options={'user_based': False}),
    'KNNBaseline': KNNBaseline(sim_options={'user_based': False})
}

In [ ]:
for algo_name, algo in item_algo.items():
  result = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
  item_res[algo_name] = result['test_rmse'].mean()

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity ma

In [ ]:
print('Item-based collaborative filtering')
for algo_name, rmse in item_res.items():
  print(f"Средний RMSE для {algo_name}: {round(rmse,4)}")

Item-based collaborative filtering
Средний RMSE для KNNBasic: 0.9076
Средний RMSE для KNNWithMeans: 0.8919
Средний RMSE для KNNBaseline: 0.8665


RMSE < 0,87 при получили при использовании алгоритма Item-based коллаборативной фильтрации **KNNBaseline**.